# Preparing Experiments

In this notebook, we will prepare several parts of our experiments including
deploying and configuring our workload, loading traces, and creating functions
to be called to generate the workloads.

In [1]:
from tqdm.auto import tqdm
import itertools

import os
import time

# my imports
from helpers import kube
from helpers import workload
from helpers import util
from helpers import request_funcs

fetching imagenet v2
resizing images


  0%|          | 0/100 [00:00<?, ?it/s]

converting to bentoml files


  0%|          | 0/100 [00:00<?, ?it/s]

extracting base64 files


  0%|          | 0/100 [00:00<?, ?it/s]

preprocessing for mobilenet


  0%|          | 0/100 [00:00<?, ?it/s]

In [2]:
# making sure kn is set up correctly
!kn service ls

NAME                    URL                                                    LATEST                        AGE    CONDITIONS   READY   REASON
autoscale-go            http://autoscale-go.default.kn.nima-dev.com            autoscale-go-00035            136d   3 OK / 3     True    
bench1                  http://bench1.default.kn.nima-dev.com                  bench1-00054                  145d   3 OK / 3     True    
bentoml-iris            http://bentoml-iris.default.kn.nima-dev.com            bentoml-iris-00165            47d    3 OK / 3     True    
bentoml-onnx-resnet50   http://bentoml-onnx-resnet50.default.kn.nima-dev.com   bentoml-onnx-resnet50-00036   47d    3 OK / 3     True    
helloworld-go           http://helloworld-go.default.kn.nima-dev.com           helloworld-go-00001           146d   3 OK / 3     True    
tfserving-mobilenetv1   http://tfserving-mobilenetv1.default.kn.nima-dev.com   tfserving-mobilenetv1-00029   47d    3 OK / 3     True    
tfserving-resnetv2      http

In [3]:
# EXP_CONFIG_NAME_DEFAULT = 'bentoml-iris-250m-512mb'
# EXP_CONFIG_NAME_DEFAULT = 'bentoml-onnx-resnet50-250m-512mb'
# EXP_CONFIG_NAME_DEFAULT = 'tfserving-resnetv2-250m-512mb'
# EXP_CONFIG_NAME_DEFAULT = 'tfserving-mobilenetv1-250m-512mb'
# exp_config_name = os.getenv("DEPLOYMENT_CONFIG_NAME", EXP_CONFIG_NAME_DEFAULT)

config_names = [
    'bentoml-iris-250m-512mb',
    'bentoml-onnx-resnet50-250m-512mb',
    'tfserving-resnetv2-250m-512mb',
    'tfserving-mobilenetv1-250m-512mb',
    'bentoml-pytorch-fashionmnist-250m-512mb',
]

request_limit_override = "'cpu=1000m,memory=1000Mi'"

for exp_config_name in config_names:
    exp_file = f"deployments/{exp_config_name}.json"

    workload_spec = util.load_json_file(exp_file)

    # override request and limit values
    workload_spec['opts']['--request'] = request_limit_override
    workload_spec['opts']['--limit'] = request_limit_override

    kn_command = kube.get_kn_command(**workload_spec)
    print('='*20)
    print(kn_command)
    print('-'*20)
    # to run the command, we can simply use:
    !{kn_command}
    # or from python:
    # get_ipython().system('{kn_command}')

# wait for changes to take effect
time.sleep(10)

kn service apply bentoml-pytorch-fashion-mnist --image ghcr.io/nimamahmoudi/bentoml-pytorch-fashion-mnist:20210614122950 \
  --limit 'cpu=1000m,memory=1000Mi' \
  --request 'cpu=1000m,memory=1000Mi' \
  --port 5000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
--------------------
Creating service 'bentoml-pytorch-fashion-mnist' in namespace 'default':

  0.050s The Route is still working to reflect the latest desired specification.
  0.170s Configuration "bentoml-pytorch-fashion-mnist" is waiting for a Revision to become ready.
 79.057s ...
 79.101s Ingress has not yet been reconciled.
 79.184s Waiting for load balancer to be ready
 79.575s Ready to serve.

Service 'bentoml-pytorch-fashion-mnist' created to latest revision 'bentoml-pytorch-fashion-mnist-00001' is available at URL:
http://bentoml-pytorch-fashion-mnist.default.kn.nima-dev.com


In [4]:
# testing the workload functions
# !python helpers/request_funcs.py